In [1]:
import sys
import numpy as np
from PNM import *
# define pi for convenience
pi = np.pi

def CreateAndSavePFM(out_path):
    width = 512
    height = 512
    numComponents = 3

    img_out = np.empty(shape=(width, height, numComponents), dtype=np.float32)
    
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = 1.0

    write_pfm(out_path, img_out)

def LoadAndSavePPM(in_path, out_path):
    img_in = read_ppm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=img_in.dtype)
    height,width,_ = img_in.shape # Retrieve height and width
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:] # Copy pixels

    write_ppm(out_path, img_out)

def LoadAndSavePFM(in_path, out_path):
    img_in = read_pfm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=img_in.dtype)
    height,width,_ = img_in.shape # Retrieve height and width
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:] # Copy pixels

    write_pfm(out_path, img_out)

def LoadPPMAndSavePFM(in_path, out_path):
    img_in = read_ppm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=np.float32)
    height,width,_ = img_in.shape
    for y in range(height):
        for x in range(width):
            img_out[y,x,:] = img_in[y,x,:]/255.0

    write_pfm(out_path, img_out)
            
def LoadPFMAndSavePPM(in_path, out_path):
    img_in = read_pfm(in_path)
    img_out = np.empty(shape=img_in.shape, dtype=np.float32)
    height,width,_ = img_in.shape
    for y in range(height):
        for x in range(width):
            img_in[y,x,:] = img_in[y,x,:] **(1/2.2)
            # img_out[y,x,:] = img_out[y,x,:] * 255.0, if the value is greater than 255, it will be 255
            img_out[y,x,:] = np.clip(img_in[y,x,:]*255.0, 0, 255)

    write_ppm(out_path, img_out.astype(np.uint8))

def create_sphere_image():
    width = 511
    height = 511
    numComponents = 3

    img_out = np.empty(shape=(width, height, numComponents), dtype=np.float32)
    # the sphere has diameter 511, the center point is the center of the image, the other part outside the sphere is black
    for y in range(height):
        for x in range(width):
            if (x - 255.5) ** 2 + (y - 255.5) ** 2 <= 255.5 ** 2:
                img_out[y, x, :] = 1.0
            else:
                img_out[y, x, :] = 0.0

    return img_out


In [2]:
graph = read_pfm('GraceCathedral/grace_latlong.pfm')

In [3]:
def median_cuts(median_cut, division = 4):
    #median cut
    
    parts = []
    for i in range(division-1):

        if parts == []:
            total = np.sum(median_cut)/3

            acc = 0
            y = median_cut.shape[0]
            x = median_cut.shape[1]
            if y > x:
                for y_1 in range(y):
                    acc += np.sum(median_cut[y_1, :])/3
                    if acc >= total / 2:
                        # divide the image into two parts
                        parts.append([median_cut[:y_1, :],(0, 0),(x-1, y_1-1)])
                        parts.append([median_cut[y_1:, :],(0, y_1),(x-1, y-1)])
                        break
            else:
                for x_1 in range(x):
                    acc += np.sum(median_cut[:, x_1])/3
                    if acc >= total / 2:
                        # divide the image into two parts
                        parts.append([median_cut[:, :x_1],(0, 0),(x_1-1, y-1)])
                        parts.append([median_cut[:, x_1:],(x_1, 0),(x-1, y-1)])
                        break

        else:
            temp = parts.pop(0)
            total = np.sum(temp[0])/3
            acc = 0
            y = temp[0].shape[0]
            x = temp[0].shape[1]
            if y > x:
                for y_1 in range(y):
                    acc += np.sum(temp[0][y_1, :])/3
                    if acc >= total / 2:
                        # divide the image into two parts
                        parts.append([temp[0][:y_1, :],(temp[1][0], temp[1][1]),(temp[2][0], temp[1][1] + y_1 - 1)])
                        parts.append([temp[0][y_1:, :],(temp[1][0], temp[1][1] + y_1),(temp[2][0], temp[2][1])])
                        break
            else:
                for x_1 in range(x):
                    acc += np.sum(temp[0][:, x_1])/3
                    if acc >= total / 2:
                        # divide the image into two parts
                        parts.append([temp[0][:, :x_1],(temp[1][0], temp[1][1]),(temp[1][0] + x_1 - 1, temp[2][1])])
                        parts.append([temp[0][:, x_1:],(temp[1][0] + x_1, temp[1][1]),(temp[2][0], temp[2][1])])
                        break
    partitions = [[x[1],x[2]] for x in parts]
    
    return partitions


                    
                    

In [4]:
def generate_median_cut_image(graph):
    median_cut = np.zeros(graph.shape)
    for y in range(graph.shape[0]):
        for x in range(graph.shape[1]):
            #intensity = np.sum(graph[y, x, :]) / 3
            median_cut[y, x, :] = np.sum(graph[y, x, :]) / 3 * np.sin(pi * y / graph.shape[0])
    return median_cut

In [5]:
def draw_median_cut(graph, division = 4):
    median_cut = generate_median_cut_image(graph)
    partitions = median_cuts(median_cut, division)
    # according to the partitions divide the image with different colors
    for i in range(len(partitions)):
        # draw the boundary of the partition
        # graph[partitions[i][0][1], partitions[i][0][0]:partitions[i][1][0], :] = np.array([0,0,1])
        # neighbor points of the boundary are also colored, 5*5
        graph[min(graph.shape[0]-1,max(partitions[i][1][1]-2,0)):min(graph.shape[0],max(partitions[i][1][1]+3,0)),partitions[i][0][0]:partitions[i][1][0], :] = np.array([0,0,1])
        # graph[partitions[i][1][1], partitions[i][0][0]:partitions[i][1][0], :] = np.array([0,0,1])
        graph[min(graph.shape[0]-1,max(partitions[i][0][1]-2,0)):min(graph.shape[0],max(partitions[i][0][1]+3,0)), partitions[i][0][0]:partitions[i][1][0], :] = np.array([0,0,1])
        # graph[partitions[i][0][1]:partitions[i][1][1], partitions[i][0][0], :] = np.array([0,0,1])
        graph[partitions[i][0][1]:partitions[i][1][1], min(graph.shape[1]-1,max(partitions[i][1][0]-2,0)):min(graph.shape[1],max(partitions[i][1][0]+3,0)), :] = np.array([0,0,1])
        # graph[partitions[i][0][1]:partitions[i][1][1], partitions[i][1][0], :] = np.array([0,0,1])
        graph[partitions[i][0][1]:partitions[i][1][1], min(graph.shape[1]-1,max(partitions[i][0][0]-2,0)):min(graph.shape[1],max(partitions[i][0][0]+3,0)), :] = np.array([0,0,1])
        # change the color of the central point of the partition
        # graph[int((partitions[i][0][1] + partitions[i][1][1])/2), int((partitions[i][0][0] + partitions[i][1][0])/2), :] = np.array([1,1,1])
        graph[min(graph.shape[0]-1,max(int((partitions[i][0][1] + partitions[i][1][1])/2)-2,0)):min(graph.shape[0],max(int((partitions[i][0][1] + partitions[i][1][1])/2)+3,0)), min(graph.shape[1]-1,max( int((partitions[i][0][0] + partitions[i][1][0])/2)-2,0)):min(graph.shape[1],max( int((partitions[i][0][0] + partitions[i][1][0])/2)+3,0)), :] = np.array([1,1,1])
    
    return graph

In [6]:
# graph = draw_median_cut(graph, 64)

# write_pfm('divided.pfm', graph)


In [7]:
partition_number = [2, 4, 16, 64 ]
for i in partition_number:
    graph = read_pfm('GraceCathedral/grace_latlong.pfm')
    t = draw_median_cut(graph, i)
    write_pfm('divided' + str(i) + '.pfm', t)
    LoadPFMAndSavePPM('divided' + str(i) + '.pfm', 'divided' + str(i) + '.ppm')

In [8]:
graph = read_pfm('GraceCathedral/grace_latlong.pfm')
black_graph = np.zeros(graph.shape, dtype = np.float32)
partitions = median_cuts(generate_median_cut_image(graph), 64)
for i in partitions:
    # calculate the average RGB color of the partition
    color = np.sum(graph[i[0][1]:i[1][1], i[0][0]:i[1][0], :], axis = (0,1)) / (i[1][1] - i[0][1]) / (i[1][0] - i[0][0])
    # change the color of the neighbor points of the central point
    black_graph[min(graph.shape[0]-1,max(int((i[0][1] + i[1][1])/2)-2,0)):min(graph.shape[0],max(int((i[0][1] + i[1][1])/2)+3,0)), min(graph.shape[1]-1,max( int((i[0][0] + i[1][0])/2)-2,0)):min(graph.shape[1],max( int((i[0][0] + i[1][0])/2)+3,0)), :] = color
write_pfm('divided.pfm', black_graph)
LoadPFMAndSavePPM('divided.pfm', 'divided.ppm')

(512, 1024, 3)


In [14]:
# load the pbrt sphere in the form of pfm and save as ppm
samplers = [8,16,32,64]
for i in samplers:
    LoadPFMAndSavePPM('./pbrt/simple_sphere_' + str(i) + '.pfm', './pbrt/simple_sphere_' + str(i) + '.ppm')
